In [1]:
import requests
import urllib
from lxml import etree
import os
from PIL import Image
from io import BytesIO

In [2]:
# 通过url获取每个图集的链接页面
def getArticleLinks(url):
    html = requests.get(url)
    Selector = etree.HTML(html.text)
    # 通过“Xpath”获取每个图集的url后缀
    # <header><h2><a href…>
    url_list = Selector.xpath('//header/h2/a/@href')
    return url_list

In [3]:
# 每页多少个图集
T = 20

In [4]:
# 获取所有页数的所有图集
today_girl = []
for i in range(1,4):
    today_girl[i*T:(i+1)*T-1] = getArticleLinks('http://zhainanba.net/category/zhainanfuli/jinrimeizi/page/%s' %(i+1))

In [5]:
# 图集数量
page = len(today_girl)
page

44

In [6]:
today_girl[2]

'http://zhainanba.net/17662.html'

In [7]:
S = len('http://zhainanba.net/')
S

21

In [8]:
today_girl[2][S:].replace(".html", '')

'17662'

In [9]:
# 通过url获取每个图集的图片链接
def getZNBPic(url):
    html = requests.get(url)
    Selector = etree.HTML(html.text)
    # 通过“Xpath”获取每个图片的url
    # <img class="alignnone" src……>
    url_pic_list = Selector.xpath('//img[@class="alignnone"]/@src')
    return url_pic_list

In [10]:
# 为每个图集创建独立文件夹，并下载图片
def download_img(url_list,page,S):
    # 该目录下创建一个downloads文件夹存放下载图片
    if not os.path.exists('downloads'):
        os.mkdir('downloads')
    root_path = os.getcwd()
    x =1
    for i in range(page):
        img_dir = 'downloads/' + url_list[i-1][S:].replace(".html", '')
        if not os.path.exists(img_dir):
            os.mkdir(img_dir)
        os.chdir(img_dir)
        Pic_Links = getZNBPic(url_list[i-1])
        L = len(Pic_Links)
        Pic_Links_X = getZNBPic(url_list[i-1])
        Pic_Links_i = []
        for i in range(0,L):
            Pic_Links_i = Pic_Links[i]
            if 'http://ac.meijiecao.net/ac/img/znb/meizitu/' not in Pic_Links_i:
                Pic_Links_X.remove(Pic_Links_i)
        for img in Pic_Links_X:
            response = requests.get(img)
            image = Image.open(BytesIO(response.content))
            image.save('zhainanba_%s.jpg' %x)
            x += 1
        os.chdir(root_path)

In [11]:
download_img(today_girl,page,S)